# ExoCTK Contamination and Visibility Tool Demo
This notebook contains general instructions on how to run the Contamination and Visibility Tool as well as some validation tests with on-sky data. The Web application for this tool can also be found at https://exoctk.stsci.edu/contam_visibility.

In [1]:
# Imports
from astropy.io import fits
from astroquery.mast import Observations
from exoctk.contam_visibility import field_simulator as fs
from exoctk.contam_visibility import contamination_figure as cf
from bokeh.plotting import show
from bokeh.io import output_notebook
from hotsoss.plotting import plot_frame
output_notebook()

pandeia not installed. Functionality limited.


**WARNING**: LOCAL JWST PRD VERSION PRDOPSSOC-067 DOESN'T MATCH THE CURRENT ONLINE VERSION PRDOPSSOC-068
Please consider updating pysiaf, e.g. pip install --upgrade pysiaf or conda update pysiaf


Loading BokehJS ...

## Using the Contamination and Visibility Tool
The Contamination and Visibility Tool has three types of outputs, discussed below.

### Simulate a field at a single position angle
The only necessary inputs to simulate the contamination in a given field at a particular position angle are the target coordinates, the instrument name, and the position angle of interest. We first search the neighborhood for sources with our RA and Dec values.

In [3]:
# Find all sources (including the target) in the field of view
target = "WASP-100"
ra, dec = 68.95970886173, -64.02703718088
sources = fs.find_sources(ra, dec, verbose=False)

INFO: Query finished. [astroquery.utils.tap.core]


Next we can calculate the contamination for a given instrument at a given PA. PA values can range from (0, 360) and supported subarrays currently include 'NIS_SUBSTRIP256' and 'NIS_SUBSTRIP96'.

In [4]:
# Calculate contamination
pa = 200
aperture = 'NIS_SUBSTRIP256'
result, plot = fs.calc_v3pa(pa, sources, aperture, plot=True, verbose=False)

# Plot it!
show(plot)

KeyError: 'trace_o1'

In the top plot, the red lines indicate stars and the pink lines indicate extended sources. The solid lines indicate the target traces and the dashed lines indicate contaminant traces. The circles indicate 0th order contaminant traces. Hover over any glyph to see more information about the source.

In the bottom plot, the shaded regions show the estimated contamination from neighboring sources in each column for each order trace.

### Simulate the estimated contamination at all position angles
If you supply the `field_simulation` function with the RA and Dec of interest, most of the work is done for you and an estimate of the contamination at all position angles is shown. This can be used to quickly see which position angle ranges are suitable for your observations.

In [ ]:
# Run the field simulation for all PAs
targframe, contamframe, results = fs.field_simulation(ra, dec, subarray, plot=False, multi=False)

In [ ]:
# Get bad PA list from missing angles between 0 and 360
badPAs = [j for j in np.arange(0, 360) if j not in [i['pa'] for i in results]]

# Make old contam plot. Some reshaping required.
simframe = np.zeros((362, 2048, 256))
simframe[0, :, :] = (targframe[0]).T[::-1, ::-1]
simframe[1, :, :] = (targframe[1]).T[::-1, ::-1]
simframe[2:, :, :] = contamframe.swapaxes(1, 2)[:, ::-1, ::-1]
contam_plot = cf.contam(simframe, subarray, targetName=target, badPAs=badPAs)
show(contam_plot)

### Explore contamination for all valid position angles
You can also simulate each field individually for a closer visual inspection with an interactive plot. We'll just use the results from above. Drag the slider to see the simulated contamination at each visible PA. The grey areas indicate position angles where the target is not visible.

In [ ]:
interactive_plot = fs.contam_slider_plot(results)
show(interactive_plot)

## Validation Tests
Below are some tests to show how well the Contamination and Visibility Tool is predicting the locations of contaminant traces. The `calc_vpa3` function also accepts on-sky SOSS data as a quality check for the output simulations.

### Download the data
First, lets's download the data we need to test the Contamination and Visibility Tool from MAST. Let's use WASP-39 as our example. Feel free to change it to your target of choice! The code accepts only a single 2D frame at a time so let's just use the 2D `_rate.fits` data product.

In [ ]:
# Specify the observation
observation = Observations.query_criteria(instrument_name='NIRISS/SOSS', filters='CLEAR;GR700XD', target_name='WASP-39')

# Get the data products
data_products = Observations.get_product_list(observation)

# Filter them to get rates only
rate_file = Observations.filter_products(data_products, dataproduct_type='spectrum', productType='SCIENCE', productSubGroupDescription='RATE')[2]

# Download the data
dl_table = Observations.download_products(rate_file)

# Get the path to the rate file
rate_path = dl_table['Local Path'][0]

### Load the data into the session
Now we can read in the data and get the target RA, Dec, and position angle we want to replicate.

In [ ]:
# Observation parameters
rate_data = fits.getdata(rate_path)
ra = fits.getval(rate_path, 'TARG_RA')
dec = fits.getval(rate_path, 'TARG_DEC')
pa = int(fits.getval(rate_path, 'GS_V3_PA'))
obsdate = fits.getval(rate_path, 'DATE-BEG')[:4]
subarray = 'NIS_' + fits.getval(rate_path, 'SUBARRAY')

### Find all nearby sources
Next, we have to search the neighborhood for our target as well as any potentially contaminanting sources. The `find_sources` function also accepts a `target_date` to specify the calendar year when the observations were taken so that proper motions of the sources can be accounted for. It is set to the current year by default but we will supply a 4-digit year from the observation `obsdate` so we know it matches.

In [ ]:
# Find all sources (including the target) in the field of view
sources = fs.find_sources(ra, dec, target_date=obsdate, verbose=False)

### Run the Contamination and Visibility Tool
Now that we have our data and observation parameters, we can run the tool.

In [ ]:
# Calculate contamination
result_onsky, plot_onsky = fs.calc_v3pa(pa, sources, subarray, data=rate_data, plot=True, verbose=False)

# Plot it!
show(plot_onsky)

In [ ]:
# For comparison, let's see what the simulation looks like
result_sim, plot_sim = fs.calc_v3pa(pa, sources, subarray, plot=True, verbose=False)

# Plot it!
show(plot_sim)